In [ ]:
import pyreadr as pyr
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import pyranges as pr
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.feature_selection import RFE
import sklearn.model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
ctcf = pyr.read_r("CTCF.rds")
stat = pyr.read_r("STAT3.rds")
ets = pyr.read_r("ETS1.rds")
myod = pyr.read_r("MYOD1.rds")
fosl = pyr.read_r("FOSL2.rds")

In [ ]:
ctcf_chip = pr.read_bed("CTCF_remap.bed", as_df=True)
stat_chip = pr.read_bed("STAT3_remap.bed", as_df=True)
ets_chip = pr.read_bed("ETS1_remap.bed", as_df=True)
myod_chip = pr.read_bed("MYOD1_remap.bed", as_df=True)
fosl_chip = pr.read_bed("FOSL2_remap.bed", as_df=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=ctcf_chip["Score"],
                         mode="lines"))

In [ ]:
ctcf = ctcf[None]
stat = stat[None]
ets = ets[None]
myod = myod[None]
fosl = fosl[None]

## 1. Precision

In [ ]:
def precision(tf, name):
    
    fig = go.Figure()
    
    filters_list = [10,20,30,40,50,60]
    
    precisions_list = []
    
    for filtr in filters_list:

        filt = tf["tfbs"].astype(int) <= filtr

        tf_masked = tf[filt]

        true_positives = tf_masked["tfbs"].value_counts().iloc[1]
        positives = tf_masked["tfbs"].count()

        precision = round((true_positives / positives) * 100, 2) 

        precisions_list.append(precision)

    fig.add_trace(go.Scatter(x=filters_list,
                             y=precisions_list))

        
    fig.update_layout(title = name)
            
    fig.show()
    print(precisions_list)

In [ ]:
precision(ctcf, "CTCF")
#precision(stat, "STAT3")
#precision(ets, "ETS1")
#precision(myod, "MYOD2")
#precision(fosl, "FOSL1")

Precision for CTCF: 34.62 %

Precision for STAT3: 1.28 %

Precision for ETS1: 1.3 %

Precision for MYOD2: 4.52 %

Precision for FOSL1: 4.41 %

The precisions do not change with increasing LLR. Model complexity is insufficient(High Bias, low variance --> underfitting). DNA sequence (motif) information alone is not enough for predicting TFBS, epigenetic modifications must be accounted for.

## 2.Correlation Analysis

### a) Violin plots with box plots

In [ ]:
def violin(data):
    
    plot_dict = {"accessibility": (1,1),
                 "DNAme": (1,2),
                 "H2az": (1,3),
                 "H3k27ac": (1,4),
                 "H3k27me3": (2,1),
                 "H3k36me3": (2,2),
                 "H3k4me1": (2,3),
                 "H3k4me2": (2,4),
                 "H3k4me3": (3,1),
                 "H3k79me2": (3,2),
                 "H3k9ac": (3,3),
                 "H3k9me3": (3,4), 
                 "H4k20me1": (4,1), 
                 "phylop": (4,2), 
                 "LLR": (4,3)}
    
    cols_list = data.columns
    
    fig = make_subplots(rows=4, cols=4, subplot_titles=cols_list )
    
    for colname in cols_list:
        
        if bool(colname == "tfbs") is True:
            
            continue
            
        else:
    
            fig.add_trace(go.Violin(y=data[colname],
                                    x=data["tfbs"],
                                    box_visible=True,
                                    meanline_visible=True,
                                    name=colname), plot_dict[colname][0], plot_dict[colname][1])

            
    fig.update_layout(title="Violin, correlations")
    fig.show()

In [ ]:
violin(ctcf)

In [ ]:
violin(stat)

In [ ]:
violin(ets)

In [ ]:
violin(myod)

In [ ]:
violin(fosl)

The base model appears to be making more precise predictions (34.63 %) for binding sites of CTCF. From the comparison of violin plots with other tfs, this should be a reason of lower epigenetic modifications for CTCF binding sites. The main distribution diffrence for predictions for CTCF is in the violin plot for DNAme. Therefore, DNMme could be a variable to add to the model.

### b) Correlation of features

In [ ]:
def correlations_heatmap(data):
    
    data = data.astype(float)
    
    correlation_list = []
    
    cols_list = data.columns
    
    for cols in cols_list:
        for cols1 in cols_list:
            
            correlation = data[cols].corr(data[cols1])
            
            correlation_list.append(correlation)
            
    correlation_array = np.asarray(correlation_list)
    
    correlation_array = correlation_array.reshape((16, 16))
    
    df = pd.DataFrame(correlation_array, index=cols_list, columns=cols_list)
    
    heatmap_dict = {'z': df.values.tolist(),
                    'x': df.columns.tolist(),
                    'y': df.index.tolist()}
    
    fig = go.Figure(data=go.Heatmap(heatmap_dict))
    fig.show()

In [ ]:
correlations_heatmap(ctcf)

In [ ]:
correlations_heatmap(stat)

In [ ]:
correlations_heatmap(ets)

In [ ]:
correlations_heatmap(myod)

In [ ]:
correlations_heatmap(fosl)

## 3) Logistic Regression Model & Variable Selection

I have built a model function (linear_regression1) with manual cross_validation steps. From the observations in the heatmaps higher correlation values are hints for potential variables to be eliminated (in case of overfitting). I selected the variables to be eliminated in the model from the strong correlation signals in the heatmaps. For ctcf the model started with 71 % of accuracy. But as it can be seen in the further steps, when functions logistic_regression and accuracy_evaluator come in the play with eliminated variables defined in them, the accuracy of ctcf did not decrease significantly for ctcf. Despite a slight increase in the precision for STAT3, the precision of the model is not as good precision of the base model, as it can make no predicitons for FOSL1, MYOD2 and ETS1

This is the same when I built a the true model (model_evaluated) accounting for the variables extracted by the evaluations steps in def model_evaluations and def model.The model was tested for it's precision and accuracy by def accuracy_evaluator_evolved 

In [ ]:
def logistic_regression1(data):
    
    """With manual cross-validation, variables selection is made with the help of heatmaps above"""
    
    data_cols = data.columns.to_series()
    data_cols = data_cols[data_cols != "tfbs"] # if we put this variable in, the model works perfectly (duh!)
    data_cols = data_cols[data_cols != "accessibility"]
    data_cols = data_cols[data_cols != "H2az"]
    data_cols = data_cols[data_cols != "H3k27ac"]
    #data_cols = data_cols[data_cols != "H3k27me3"]
    data_cols = data_cols[data_cols != "H3k36me3"]
    #data_cols = data_cols[data_cols != "H3k4me1"]
    data_cols = data_cols[data_cols != "H3k4me2"]
    data_cols = data_cols[data_cols != "H3k4me3"]
    data_cols = data_cols[data_cols != "H3k79me2"]
    #data_cols = data_cols[data_cols != "H3k9ac"]
    #data_cols = data_cols[data_cols != "H3k9me3"]
    #data_cols = data_cols[data_cols != "H4k20me1"]
    #data_cols = data_cols[data_cols != "phylop"]
    #data_cols = data_cols[data_cols != "DNAme"]
    #data_cols = data_cols[data_cols != "LLR"]
    
    data_cols = data_cols.values
    
    X = data[data_cols]
    y = data["tfbs"]
    
    #  manual cross-validation
    
    X1_train,X1_test,y1_train,y1_test = train_test_split(X,y,test_size=0.25,random_state=0)
    X2_train,X2_test,y2_train,y2_test = train_test_split(X,y,test_size=0.10,random_state=1)
    X3_train,X3_test,y3_train,y3_test = train_test_split(X,y,test_size=0.15,random_state=2)
    X4_train,X4_test,y4_train,y4_test = train_test_split(X,y,test_size=0.05,random_state=3)
    #X5_train,X5_test,y5_train,y5_test = train_test_split(X,y,test_size=0.25,random_state=4)
    
    logistic_regression = LogisticRegression()
    
    logistic_regression.fit(X1_train,y1_train)
    
    #sklearn.model_selection.cross_validate(logistic_regression, X, y) # default = 5-Fold
    
    logistic_regression.fit(X2_train,y2_train)
    logistic_regression.fit(X3_train,y3_train)
    logistic_regression.fit(X4_train,y4_train)
    #logistic_regression.fit(X5_train,y5_train)
    
    return logistic_regression

In [ ]:
model = logistic_regression1(ctcf)

In [ ]:
def accuracy_evaluator(data, model):
    
    data_cols = data.columns.to_series()
    data_cols = data_cols[data_cols != "tfbs"]
    data_cols = data_cols[data_cols != "accessibility"]
    data_cols = data_cols[data_cols != "H2az"]
    data_cols = data_cols[data_cols != "H3k27ac"]
    #data_cols = data_cols[data_cols != "H3k27me3"]
    data_cols = data_cols[data_cols != "H3k36me3"]
    #data_cols = data_cols[data_cols != "H3k4me1"]
    data_cols = data_cols[data_cols != "H3k4me2"]
    data_cols = data_cols[data_cols != "H3k4me3"]
    data_cols = data_cols[data_cols != "H3k79me2"]
    #data_cols = data_cols[data_cols != "H3k9ac"]
    #data_cols = data_cols[data_cols != "H3k9me3"]
    #data_cols = data_cols[data_cols != "H4k20me1"]
    #data_cols = data_cols[data_cols != "phylop"]
    #data_cols = data_cols[data_cols != "DNAme"]
    #data_cols = data_cols[data_cols != "LLR"]
    
    data_cols = data_cols.values
    
    X = data[data_cols]
    y = data["tfbs"]
    
    y_pred = model.predict(X)
    
    confusion_matrix = pd.crosstab(y, y_pred, rownames=["Actual"], colnames=["Predicted"])
    
    print('Accuracy: ',metrics.accuracy_score(y, y_pred))
    print('Precision: ',metrics.precision_score(y, y_pred, pos_label="1"))

In [ ]:
accuracy_evaluator(ctcf, model)

In [ ]:
accuracy_evaluator(stat, model) # note the manual cross validation steps.

In [ ]:
accuracy_evaluator(ets, model) # result of overfitting(?), No predicitions are made by the model for ETS1 data.

In [ ]:
#accuracy_evaluator(myod, model)

In [ ]:
#accuracy_evaluator(fosl, model)

In [ ]:
#accuracy_evaluator(ctcf, model2)

In [ ]:
#accuracy_evaluator(stat, model2)

## 4) Using RFE and Cross-Validation from sklearn.


In [ ]:
def model_evaluations(data):
    
    """Cross Validation, RFE, using sklearn and all variables"""
    
    data_cols = data.columns.to_series()
    data_cols = data_cols[data_cols != "tfbs"] 
    
    data_cols = data_cols.values
    
    X = data[data_cols]
    y = data["tfbs"]
    
    models = dict()
    
    for i in range(2, 17):
        selection = RFE(estimator=LogisticRegression(), n_features_to_select=i)
        logistic_regression = LogisticRegression()
        models[str(i)] = Pipeline(steps=[('s',selection),('m',logistic_regression)])
    
    # cross-validation
    
    def evaluate_model(model, X, y): # split = 0.20
        cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
        scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        return scores
        
    results, names = list(), list()
    for name, model in models.items():
        scores = evaluate_model(model, X, y)
        results.append(scores)
        names.append(name)
        print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

            
    pyplot.boxplot(results, labels=names, showmeans=True)
    pyplot.show()

In [ ]:
model_stat = model_evaluations(stat)

In [ ]:
model_ets = model_evaluations(ets)

In [ ]:
model_myod = model_evaluations(myod)

In [ ]:
model_fosl = model_evaluations(fosl)

In [ ]:
model_ctcf = model_evaluations(ctcf)

## ---->>  12  Variables to be seleceted for the model trained on CTCF data.

The model cannot be trained by using other tf data ---> low rate of true positives may be an explanation

In [ ]:
def model(data, variable_count):
    
    """Cross Validation, RFE, using sklearn and all variables"""
    
    data_cols = data.columns.to_series()
    data_cols = data_cols[data_cols != "tfbs"] 
    #data_cols = data_cols[data_cols != "accessibility"]
    #data_cols = data_cols[data_cols != "H2az"]
    #data_cols = data_cols[data_cols != "H3k27ac"]
    #data_cols = data_cols[data_cols != "H3k27me3"]
    #data_cols = data_cols[data_cols != "H3k36me3"]
    #data_cols = data_cols[data_cols != "H3k4me1"]
    #data_cols = data_cols[data_cols != "H3k4me2"]
    #data_cols = data_cols[data_cols != "H3k4me3"]
    #data_cols = data_cols[data_cols != "H3k79me2"]
    #data_cols = data_cols[data_cols != "H3k9ac"]
    #data_cols = data_cols[data_cols != "H3k9me3"]
    #data_cols = data_cols[data_cols != "H4k20me1"]
    #data_cols = data_cols[data_cols != "phylop"]
    #data_cols = data_cols[data_cols != "DNAme"]
    #data_cols = data_cols[data_cols != "LLR"]
    
    data_cols = data_cols.values
    
    X = data[data_cols]
    y = data["tfbs"]
    
    model = LogisticRegression()
    
    selection = RFE(estimator=LogisticRegression(), n_features_to_select=variable_count)
    

    pipeline = Pipeline(steps=[("s",selection),("m",model)])
    pipeline.fit(X, y)
    
    
    sklearn.model_selection.cross_validate(pipeline, X, y) # default: k = 5
    
    for i in range(X.shape[1]):
        print("Column: %d, Selected %s, Rank: %.3f" % (i, selection.support_[i], selection.ranking_[i]))

    
    return model

In [ ]:
model_improved = model(ctcf, 12)

In [ ]:
def accuracy_evaluator_evolved(data, model):
    
    data_cols = data.columns.to_series()
    data_cols = data_cols[data_cols != "tfbs"]
    #data_cols = data_cols[data_cols != "accessibility"] #0
    #data_cols = data_cols[data_cols != "H2az"]
    #data_cols = data_cols[data_cols != "H3k27ac"]
    data_cols = data_cols[data_cols != "H3k27me3"]
    #data_cols = data_cols[data_cols != "H3k36me3"] #4
    #data_cols = data_cols[data_cols != "H3k4me1"]
    data_cols = data_cols[data_cols != "H3k4me2"]
    #data_cols = data_cols[data_cols != "H3k4me3"]
    #data_cols = data_cols[data_cols != "H3k79me2"]
    data_cols = data_cols[data_cols != "H3k9ac"] #9
    data_cols = data_cols[data_cols != "H3k9me3"]
    #data_cols = data_cols[data_cols != "H4k20me1"]
    #data_cols = data_cols[data_cols != "phylop"]
    #data_cols = data_cols[data_cols != "DNAme"]
    #data_cols = data_cols[data_cols != "LLR"] #14
    
    data_cols = data_cols.values
    
    X = data[data_cols]
    y = data["tfbs"]
    
    y_pred = model.predict(X)
    
    confusion_matrix = pd.crosstab(y, y_pred, rownames=["Actual"], colnames=["Predicted"])
    
    print('Accuracy: ',metrics.accuracy_score(y, y_pred))
    print('Precision: ',metrics.precision_score(y, y_pred, pos_label="1"))

In [ ]:
accuracy_evaluator_evolved(ctcf, model_improved) #increase in precision of the model by 30 % 

In [ ]:
accuracy_evaluator_evolved(stat, model_improved) #increase in precision of the model by 6 % 

In [ ]:
accuracy_evaluator_evolved(ets, model_improved) # same error for the rest

## Conclusion

Increasing variable numbers seem to improve precision for CTCF and STAT3 binding sites until variable_count = 12. Although the trade-off between variance and bias is accounted for in the evaluation of boxplots (to prohibit overfitting) the model is still lacking the ability of finding tfbs for ETS1, MYOD1 and FOSL2. The model cannot be generalized. 

In [ ]:
pip --allow-chromium-download